# Split.ipynb

Splits up the tilt-schema json into its constituent parts. 
It further moves the properties section of the tilt into another subdirectory and splits it again. 

You should only execute the cells in this notebook if you want to definitely overwrite the files that are in the tilt_schema directory. 

These files are used by the GitHub action in main.yaml to reconstruct the tilt. 

This decomposition was made for easier readability and tractability of the tilt-schema

In [184]:
import json
import os
import glob
import time

# adopted from chatGPT

def split_json(filepath_og):
    # Open the original JSON file
    with open(filepath_og, 'r') as f:
        data = json.load(f)

    # Iterate over the top-level keys in the JSON object
    for i, key in enumerate(data.keys()):
        # Create a new dictionary containing only the data for this key
        section = { key: data[key] }

        # Write the section to a new JSON file
        with open(f'{i:02d}_{key}.json', 'w') as f:
            json.dump(section, f)

    return data.keys()

split_json('../tilt-schema.json')


# Open the original JSON file
with open('06_properties.json', 'r') as f:
    data = json.load(f)

# Iterate over the top-level keys in the JSON object
for i, key in enumerate(data['properties'].keys()):
    # Create a new dictionary containing only the data for this key
    section = { key: data['properties'][key] }

    # Write the section to a new JSON file
    os.makedirs(os.path.dirname(f'properties/{i:02d}_{key}.json'), exist_ok=True)
    with open(f'properties/{i:02d}_{key}.json', 'w') as f:
        json.dump(section, f)

os.remove("06_properties.json")

In [185]:
def reconstruct():

    """
    func:   Constructs the tilt-schema out of a number of constituent parts.


    """

    
    properties_sections = sorted(glob.glob('properties/*.json'))

    
    print(properties_sections)
    
    # Update the properties file with the underlying values 
    # Note: presupposes that all changes to the properties have been made in the properties-folder
    data_props = []
    for subfile in properties_sections:

        with open(subfile, 'r') as sf:
            data_props.append(json.load(sf))
            
    combo_props = {}
    for item in data_props:
        combo_props.update(item)
        
    # Write the combined data to a the properties file
    with open('06_properties.json', 'w') as f:
        json.dump({'properties':combo_props}, f)
    
    meta_sections = sorted(glob.glob("*.json"))
    print(meta_sections)


    # Create an empty list to hold the data from each file
    data = []

    # Iterate over each JSON file and load its contents into the data list
    for file in meta_sections:
        with open(file, 'r') as f:
            data.append(json.load(f))

    # Combine all the data into a single dictionary
    combined_data = {}
    for item in data:
        combined_data.update(item)

    
    # Write the combined data to a new JSON file
    with open('../combined.json', 'w') as f:
        json.dump(combined_data, f)
        print('successfully created new json')

    os.remove("06_properties.json")

    return

In [186]:
reconstruct()

['properties/00_meta.json', 'properties/01_controller.json', 'properties/02_dataProtectionOfficer.json', 'properties/03_dataDisclosed.json', 'properties/04_thirdCountryTransfers.json', 'properties/05_accessAndDataPortability.json', 'properties/06_sources.json', 'properties/07_rightToInformation.json', 'properties/08_rightToRectificationOrDeletion.json', 'properties/09_rightToDataPortability.json', 'properties/10_rightToWithdrawConsent.json', 'properties/11_rightToComplain.json', 'properties/12_automatedDecisionMaking.json', 'properties/13_changesOfPurpose.json']
['00_$schema.json', '01_$id.json', '02_type.json', '03_title.json', '04_description.json', '05_examples.json', '06_properties.json', '07_required.json']
successfully created new json


In [187]:
# Load the data from the first file
with open('../tilt-schema.json', 'r') as f:
    data1 = json.load(f)

# Load the data from the second file
with open('../combined.json', 'r') as f:
    data2 = json.load(f)

# Compare the two data sets and print out any differences
if data1 != data2:
    print("Differences found between the two files:")
    for key in set(data1.keys()).union(set(data2.keys())):
        if data1.get(key) != data2.get(key):
            print(f"Key '{key}' differs between the files:")
            print(f"File 1: {data1.get(key)}")
            print(f"File 2: {data2.get(key)}")
else:
    print("No differences found between the two files.")

No differences found between the two files.


In [188]:
print(len(str(data1)))
print(len(str(data2)))

63466
63466
